## 파인콘 벡터 DB 저장


In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader

## 1. 문서 내용 읽고 분할 
loader = Docx2txtLoader('law_2.docx')

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
document_list = loader.load_and_split(text_splitter=text_spliter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장

## 2.1. 환경변수 읽어오기
load_dotenv

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [2]:
from pinecone import Pinecone
import os

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

c:\Users\71\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_pinecone import PineconeVectorStore

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name='law-index2',
)

database

In [4]:
len(document_list)

5

In [5]:
## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색

## 3.1. 사용자 질문
query = '전세사기 피해자 지원 방법은?'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

## 4.0 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

## 4.1. 프롬프트 작성

prompt = '''
[identity]
-당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query,
)

## 4.3. LLM 모델 생성(chatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM  모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기피해자는 여러 가지 방법으로 지원을 받을 수 있습니다. 아래는 전세사기피해자 지원 및 주거안정에 관한 특별법 시행령에 따라 제공되는 주요 지원 방법들입니다:\n\n1. **전세사기피해지원단 구성 및 운영**: 국토교통부에서 전세사기피해지원단을 구성하여 피해자 지원을 총괄하는 역할을 합니다.\n\n2. **국세 및 지방세 안분 신청**: 피해자는 안분 신청을 통해 임대인의 국세 및 지방세 납부 부담을 줄일 수 있습니다. 전세사기피해주택의 임대인이 보유한 모든 주택의 가격과 전세사기피해주택의 시가표준액을 기준으로 안분 금액이 산정됩니다.\n\n3. **전세사기피해주택 매입 및 우선 공급**: 공공주택 사업자가 경매나 공매를 통해 전세사기피해주택을 매입하여 피해자에게 우선 공급할 수 있도록 지원합니다. 이때 임대료나 매입 차액에 대해 재정 지원도 이루어질 수 있습니다.\n\n4. **공공임대주택 우선 공급 및 임대료 지원**: 피해자가 공공임대주택에 거주할 경우, 임대료 감면이 제공되며, 감면된 임대료에 대해서는 국가 또는 지방자치단체가 재정 지원을 통해 부담을 완화합니다.\n\n5. **전세피해지원센터 운영 및 정보 제공**: 국토교통부장관이 설치한 전세피해지원센터를 통해 피해자에게 필요한 정보와 지원을 제공하며, 관련 데이터를 체계적으로 관리합니다.\n\n이러한 방법들이 전세사기 피해자들에게 실질적인 도움을 주기 위해 마련되어 있습니다. 구체적인 지원을 받기 위해서는 국토교통부나 전세피해지원센터에 문의하셔서 추가적인 정보를 얻으실 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 437, 'prompt_tokens': 3894, 'total_tokens': 4331, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio